# SOLAR-10.7B 한국어 WMS 파인튜닝 (KT Cloud H100E 최적화)

이 노트북은 Upstage의 SOLAR-10.7B 모델을 KT Cloud H100E 환경에서 한국어 WMS(창고관리시스템) 도메인에 특화하여 파인튜닝하는 전체 과정을 담고 있습니다.

### 주요 단계
1. **환경 설정**: 필요한 라이브러리 설치 및 환경 체크
2. **데이터 준비**:
   - 대규모 한국어 데이터셋 로드
   - Gemini 1.5 Flash API를 활용한 WMS 전문 데이터 생성
3. **모델 설정**: SOLAR-10.7B 모델 로딩 및 LoRA 설정
4. **훈련**: KT Cloud H100E에 최적화된 설정으로 모델 파인튜닝
5. **평가 및 업로드**: (옵션) 훈련된 모델 테스트 및 Hugging Face Hub에 업로드

### 최적화 환경
- **GPU**: NVIDIA H100E
- **Framework**: PyTorch 2.6, CUDA 12.8
- **Techniques**: Flash Attention 2, BF16, Fused AdamW, LoRA


In [28]:
# <<< 반드시 가장 첫 셀에 실행 >>>
import os, sys
base = "/home/work"  # 쓰기 가능한 경로

pkg_root = os.path.join(base, "transformer_engine")
pkg_common = os.path.join(pkg_root, "common")

os.makedirs(pkg_common, exist_ok=True)

# 빈 모듈들 생성: __init__.py, pytorch.py, common/__init__.py
open(os.path.join(pkg_root, "__init__.py"), "w").close()
open(os.path.join(pkg_root, "pytorch.py"), "w").close()
open(os.path.join(pkg_common, "__init__.py"), "w").close()

# 이 경로를 파이썬 검색 경로 최우선으로 (시스템 TE 대신 우리가 만든 더미를 잡게)
if base not in sys.path:
    sys.path.insert(0, base)

# (권장) device_map 환경변수 오염 방지 + 단일 GPU 고정
for k in ("ACCELERATE_USE_DEVICE_MAP", "HF_ACCELERATE_USE_DEVICE_MAP"):
    os.environ.pop(k, None)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print("환경 안정화 완료!")

환경 안정화 완료!


## 2. 기본 설정 및 임포트

파인튜닝에 필요한 모든 라이브러리를 임포트하고, 기본 설정을 정의합니다.
- `KTCloudH100Config` 클래스를 통해 모든 하이퍼파라미터와 설정을 관리합니다.
- Gemini API 키, Hugging Face 사용자명 등은 이 단계에서 설정할 수 있습니다.


In [29]:
# 필수 패키지 설치
!pip install --user -U \
"transformers>=4.45.0" \
"peft==0.17.1" \
"accelerate>=0.34.0" \
"bitsandbytes>=0.43.1" \
"safetensors>=0.4.4" \
"datasets>=2.0.0" \
"huggingface_hub>=0.16.0"

print("패키지 설치 완료! 커널을 재시작해주세요.")

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1

In [30]:
import os
import json
import torch
import random
from datetime import datetime
from typing import List, Dict, Optional, Any
from dataclasses import dataclass

# Transformers and PEFT
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training,
    TaskType,
    PeftModel
)

# Datasets
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

print("모든 라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 디바이스: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU 메모리: {gpu_memory:.1f}GB")
    
    # H100 확인
    gpu_name = torch.cuda.get_device_name(0)
    if "H100" in gpu_name:
        print("H100 감지! 최적화 모드 활성화")

모든 라이브러리 임포트 완료!
PyTorch 버전: 2.6.0a0+ecf3bae40a.nv25.01
CUDA 사용 가능: True
GPU 디바이스: NVIDIA H100 80GB HBM3
GPU 메모리: 79.2GB
H100 감지! 최적화 모드 활성화


In [22]:
@dataclass
class SOLARKoreanConfig:
    """SOLAR-10.7B 한국어 파인튜닝 설정"""
    
    # Model settings
    base_model: str = "upstage/SOLAR-10.7B-v1.0"
    model_name: str = "SOLAR-10.7B-Korean-Instruct"
    
    # Training settings
    output_dir: str = "/home/work/solar-korean-output"
    num_train_epochs: int = 3
    per_device_train_batch_size: int = 8  # H100E에서 안전한 배치 크기
    gradient_accumulation_steps: int = 2  # effective batch size = 16
    learning_rate: float = 2e-5
    weight_decay: float = 0.01
    warmup_ratio: float = 0.03
    max_length: int = 2048
    
    # LoRA settings - SOLAR에 최적화
    lora_r: int = 32
    lora_alpha: int = 64
    lora_dropout: float = 0.1
    
    # Hardware settings
    use_4bit: bool = False  # H100은 메모리 충분
    use_bf16: bool = True
    device_map: str = "auto"
    
    # Performance settings
    dataloader_num_workers: int = 8
    gradient_checkpointing: bool = True
    
# 설정 인스턴스 생성
config = SOLARKoreanConfig()

print("설정 완료!")
print(f"기반 모델: {config.base_model}")
print(f"출력 디렉토리: {config.output_dir}")
print(f"배치 크기: {config.per_device_train_batch_size} x {config.gradient_accumulation_steps} = {config.per_device_train_batch_size * config.gradient_accumulation_steps}")
print(f"LoRA 설정: r={config.lora_r}, alpha={config.lora_alpha}")

# 출력 디렉토리 생성
os.makedirs(config.output_dir, exist_ok=True)

설정 완료!
기반 모델: upstage/SOLAR-10.7B-v1.0
출력 디렉토리: /home/work/solar-korean-output
배치 크기: 8 x 2 = 16
LoRA 설정: r=32, alpha=64


In [31]:
# 데이터 파일 확인 및 로드
data_path = "/home/work/tesseract/korean_base_dataset.json"

if os.path.exists(data_path):
    print(f"데이터 파일 로드 중: {data_path}")
    
    with open(data_path, 'r', encoding='utf-8') as f:
        korean_data = json.load(f)
    
    print(f"총 데이터 개수: {len(korean_data):,}개")
    
    # 데이터 구조 확인
    sample = korean_data[0]
    print("\n데이터 구조:")
    print(f"- instruction: {len(sample.get('instruction', ''))}")
    print(f"- output: {len(sample.get('output', ''))}")
    print(f"- messages: {len(sample.get('messages', []))}")
    
    # 전처리 함수
    def preprocess_korean_data(data_list):
        """한국어 데이터를 chat 형태로 전처리"""
        processed = []
        
        for item in data_list:
            # messages 필드가 있으면 사용, 없으면 instruction-output으로 생성
            if 'messages' in item and item['messages']:
                messages = item['messages']
            else:
                messages = [
                    {"role": "user", "content": item['instruction']},
                    {"role": "assistant", "content": item['output']}
                ]
            
            processed.append({
                "messages": messages,
                "source": "korean_base"
            })
        
        return processed
    
    # 전처리 실행
    print("\n데이터 전처리 중...")
    processed_data = preprocess_korean_data(korean_data)
    
    # 훈련/검증 분할 (90%/10%)
    train_size = int(0.9 * len(processed_data))
    train_data = processed_data[:train_size]
    eval_data = processed_data[train_size:]
    
    print(f"훈련 데이터: {len(train_data):,}개")
    print(f"검증 데이터: {len(eval_data):,}개")
    
    # Dataset 객체 생성
    train_dataset = Dataset.from_list(train_data)
    eval_dataset = Dataset.from_list(eval_data)
    
    print("\n데이터 전처리 완료!")
    
else:
    print(f"데이터 파일을 찾을 수 없습니다: {data_path}")
    train_dataset = None
    eval_dataset = None

데이터 파일 로드 중: /home/work/tesseract/korean_base_dataset.json
총 데이터 개수: 173,785개

데이터 구조:
- instruction: 32
- output: 342
- messages: 2

데이터 전처리 중...
훈련 데이터: 156,406개
검증 데이터: 17,379개

데이터 전처리 완료!


## 3. KT Cloud 환경 체크

현재 실행 환경이 H100E에 최적화되어 있는지 확인합니다.
- PyTorch, CUDA 버전 체크
- GPU 종류 및 메모리 확인
- Flash Attention 지원 여부 확인


In [32]:
print(f"SOLAR 모델 로딩: {config.base_model}")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model,
    trust_remote_code=True
)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print("토크나이저 로드 완료")

# 모델 로드 (H100은 메모리가 충분하므로 양자화 없이)
if not config.use_4bit:
    # Full precision 로드
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model,
        device_map=config.device_map,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_cache=False  # 훈련 시 메모리 절약
    )
    print("SOLAR 모델 로드 완료 (Full precision)")
else:
    # 양자화 버전 (필요시)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model,
        quantization_config=quantization_config,
        device_map=config.device_map,
        trust_remote_code=True
    )
    print("SOLAR 모델 로드 완료 (4-bit 양자화)")

print(f"모델 파라미터 수: {model.num_parameters():,}")

SOLAR 모델 로딩: upstage/SOLAR-10.7B-v1.0
토크나이저 로드 완료


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

SOLAR 모델 로드 완료 (Full precision)
모델 파라미터 수: 10,731,524,096


## 4. Gemini API를 활용한 WMS 데이터 생성

Gemini 1.5 Flash 모델을 사용하여 WMS 도메인에 특화된 고품질 질의응답 데이터셋을 생성합니다.
- **API 키 로테이션**: 4개의 API 키를 번갈아 사용하여 API 제한(rate limit)을 최소화합니다.
- **비동기 처리**: `asyncio`를 사용하여 데이터 생성을 병렬로 처리하고 속도를 높입니다.
- **전문 프롬프트**: 재고관리, 물류운영, WMS 기술 등 세분화된 프롬프트를 사용하여 데이터의 전문성을 높입니다.


In [36]:
# 일곱 번째 셀 수정버전
# QLoRA/LoRA 준비 - gradient checkpointing 비활성화 버전
if config.use_4bit:
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
    print("4-bit 모델 훈련 준비 완료")

# SOLAR에 최적화된 LoRA 설정  
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=config.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    use_rslora=False  # RSLoRA도 비활성화해서 안정성 확보
)

print("LoRA 어댑터 적용 중...")
model = get_peft_model(model, lora_config)

# Enable gradient for LoRA parameters explicitly
model.train()
for name, param in model.named_parameters():
    if param.requires_grad:
        param.retain_grad()  # 명시적으로 gradient 유지

model.print_trainable_parameters()
print("LoRA 설정 완료!")

LoRA 어댑터 적용 중...


/home/work/.local/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/work/.local/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 125,829,120 || all params: 10,857,353,216 || trainable%: 1.1589
LoRA 설정 완료!


## 5. 한국어 데이터 로더

SOLAR 모델 파인튜닝에 사용할 대규모 한국어 데이터셋을 로드하고 전처리합니다.
- **다양한 데이터셋 활용**: KoAlpaca (Instruction), Ko-Ultrachat (대화), KULLM (고품질 대화) 등 여러 데이터셋을 통합하여 모델의 한국어 능력을 종합적으로 향상시킵니다.
- **병렬 처리**: `num_proc` 옵션을 사용하여 데이터 전처리 속도를 높입니다.
- **일관된 형식**: 모든 데이터를 `messages` 형식으로 통일하여 훈련 데이터 구조를 일관성 있게 유지합니다.


In [37]:
def tokenize_function(examples):
    """데이터셋 토크나이징 함수"""
    batch_input_ids = []
    batch_attention_masks = []
    batch_labels = []
    
    for messages in examples['messages']:
        # Chat template 적용
        try:
            formatted_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
        except:
            # Chat template이 없으면 직접 포맷
            formatted_text = ""
            for msg in messages:
                if msg['role'] == 'user':
                    formatted_text += f"사용자: {msg['content']}\n"
                elif msg['role'] == 'assistant':
                    formatted_text += f"어시스턴트: {msg['content']}\n"
        
        # 토크나이징
        tokenized = tokenizer(
            formatted_text,
            truncation=True,
            padding="max_length",
            max_length=config.max_length,
            return_tensors="pt"
        )
        
        input_ids = tokenized["input_ids"].squeeze(0)
        attention_mask = tokenized["attention_mask"].squeeze(0)
        
        # 레이블 = input_ids (causal LM)
        labels = input_ids.clone()
        
        batch_input_ids.append(input_ids)
        batch_attention_masks.append(attention_mask)
        batch_labels.append(labels)
    
    return {
        "input_ids": batch_input_ids,
        "attention_mask": batch_attention_masks,
        "labels": batch_labels
    }

print("데이터셋 토크나이징 중...")
if train_dataset is not None:
    # 토크나이징 실행
    train_tokenized = train_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=32,
        num_proc=4,
        remove_columns=train_dataset.column_names
    )
    
    eval_tokenized = eval_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=32,
        num_proc=4,
        remove_columns=eval_dataset.column_names
    )
    
    print(f"토크나이징 완료!")
    print(f"훈련 샘플: {len(train_tokenized)}")
    print(f"검증 샘플: {len(eval_tokenized)}")
else:
    print("데이터셋이 없어 토크나이징을 건너뜁니다.")

데이터셋 토크나이징 중...


Map (num_proc=4):   0%|          | 0/156406 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/17379 [00:00<?, ? examples/s]

토크나이징 완료!
훈련 샘플: 156406
검증 샘플: 17379


## 6. SOLAR 최적화 트레이너

본격적인 모델 훈련을 담당하는 `SOLARTrainer` 클래스입니다.
- **모델 및 토크나이저 설정**: `setup_model`
  - H100E 환경에 맞춰 `bfloat16`과 `Flash Attention 2`를 사용하여 모델을 로드합니다.
  - `torch.compile`을 적용하여 PyTorch 2.x의 최신 성능 최적화를 활용합니다.
  - `LoRA` 설정을 통해 효율적인 파인튜닝을 준비합니다.
- **데이터셋 전처리**: `process_dataset`
  - 모든 텍스트 데이터를 모델이 이해할 수 있는 토큰 ID로 변환합니다.
- **훈련 실행**: `train`
  - `TrainingArguments`를 통해 H100E에 최적화된 하이퍼파라미터를 설정합니다.
  - `Trainer`를 사용하여 실제 훈련을 진행하고, 최종 모델을 저장합니다.


In [ ]:
if train_dataset is not None:
    # 모델 trainable 파라미터 확인 및 수정
    print("훈련 가능한 파라미터 상태 확인...")
    
    # PEFT 모델의 경우 훈련 모드 명시적 설정
    model.train()
    
    # LoRA 파라미터가 제대로 훈련 가능한지 확인
    trainable_params = 0
    all_params = 0
    for param in model.parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    
    print(f"훈련 가능한 파라미터: {trainable_params:,} / {all_params:,} ({trainable_params/all_params*100:.2f}%)")
    
    if trainable_params == 0:
        print("❌ 훈련 가능한 파라미터가 없습니다! LoRA 설정을 다시 확인하세요.")
        # LoRA 파라미터 강제 활성화
        for name, param in model.named_parameters():
            if 'lora_' in name:
                param.requires_grad = True
                print(f"LoRA 파라미터 활성화: {name}")
    
    # 훈련 인자 설정 (API 변경사항 반영)
    training_args = TrainingArguments(
        output_dir=config.output_dir,
        num_train_epochs=config.num_train_epochs,
        per_device_train_batch_size=config.per_device_train_batch_size,
        per_device_eval_batch_size=config.per_device_train_batch_size,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        learning_rate=config.learning_rate,
        weight_decay=config.weight_decay,
        warmup_ratio=config.warmup_ratio,
        
        # H100 최적화
        bf16=config.use_bf16,
        gradient_checkpointing=config.gradient_checkpointing,
        dataloader_num_workers=config.dataloader_num_workers,
        dataloader_pin_memory=True,
        
        # 로깅 및 저장
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=500,
        save_steps=1000,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        
        # 성능 최적화
        remove_unused_columns=False,
        optim="adamw_torch_fused",  # PyTorch 2.6 최적화
        max_grad_norm=1.0,
        
        # 로깅
        report_to=[],  # 빈 리스트로 설정
        logging_dir=f"{config.output_dir}/logs",
        
        # 추가 안정성 설정
        save_safetensors=True,
        seed=42
    )
    
    # 데이터 콜레이터
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # Causal LM이므로 MLM 사용하지 않음
    )
    
    # 트레이너 생성 (API 변경사항 반영)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=eval_tokenized,
        processing_class=tokenizer,  # tokenizer 대신 processing_class 사용
        data_collator=data_collator,
    )
    
    # 추가 검증: 첫 번째 배치로 forward pass 테스트
    print("첫 번째 배치로 forward pass 테스트...")
    try:
        sample_batch = next(iter(trainer.get_train_dataloader()))
        sample_batch = {k: v.to(model.device) if hasattr(v, 'to') else v for k, v in sample_batch.items()}
        
        with torch.no_grad():
            outputs = model(**sample_batch)
            print(f"✅ Forward pass 성공! Loss: {outputs.loss:.4f}")
    except Exception as e:
        print(f"❌ Forward pass 실패: {e}")
        print("데이터 또는 모델 설정을 확인하세요.")
    
    print("SOLAR 한국어 파인튜닝 시작!")
    print(f"총 스텝: {len(train_tokenized) // (config.per_device_train_batch_size * config.gradient_accumulation_steps) * config.num_train_epochs}")
    print(f"예상 시간: 약 6-8시간 (H100)")
    
    # 훈련 실행
    try:
        trainer.train()
        print("훈련 완료!")
    except Exception as e:
        print(f"훈련 중 오류 발생: {e}")
        print("모델 상태와 데이터를 다시 확인하세요.")
        
else:
    print("데이터셋이 없어 훈련을 건너뜁니다.")

훈련 가능한 파라미터 상태 확인...
훈련 가능한 파라미터: 125,829,120 / 10,857,353,216 (1.16%)
첫 번째 배치로 forward pass 테스트...


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


✅ Forward pass 성공! Loss: 1.9100
SOLAR 한국어 파인튜닝 시작!
총 스텝: 29325
예상 시간: 약 6-8시간 (H100)


Step,Training Loss,Validation Loss


## 7. 통합 파이프라인 실행

이제 위에서 정의한 모든 컴포넌트(데이터 로더, 데이터 생성기, 트레이너)를 하나로 묶어 전체 파인튜닝 파이프라인을 실행합니다.

- **실행 순서**:
  1. 환경 체크
  2. 한국어 데이터 로드
  3. (API 키가 있을 경우) Gemini로 WMS 데이터 생성
  4. 데이터셋 통합
  5. 모델 설정 및 훈련

- **실행 방법**:
  - 아래 셀을 실행하면 전체 파이프라인이 동작합니다.
  - `config` 객체의 값을 수정하여 훈련 설정을 변경할 수 있습니다.


In [ ]:
if train_dataset is not None:
    # 모델 저장
    final_path = os.path.join(config.output_dir, "final")
    print(f"모델 저장 중: {final_path}")
    
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)
    
    print(f"모델 저장 완료: {final_path}")
    
    # 간단한 테스트
    def generate_korean_response(prompt, max_length=200):
        """한국어 응답 생성 함수"""
        # Chat format으로 입력 구성
        messages = [
            {"role": "user", "content": prompt}
        ]
        
        try:
            formatted_text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
        except:
            formatted_text = f"사용자: {prompt}\n어시스턴트:"
        
        inputs = tokenizer(formatted_text, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 프롬프트 부분 제거하고 응답만 추출
        if "어시스턴트:" in response:
            return response.split("어시스턴트:")[-1].strip()
        else:
            return response[len(formatted_text):].strip()
    
    # 테스트 질문들
    test_prompts = [
        "안녕하세요! 자기소개를 해주세요.",
        "Python과 JavaScript의 차이점을 설명해주세요.",
        "건강한 식단을 위한 조언을 해주세요.",
        "효과적인 학습 방법에 대해 알려주세요."
    ]
    
    print("\n=== SOLAR 한국어 모델 테스트 ===")
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n[테스트 {i}]")
        print(f"질문: {prompt}")
        
        try:
            response = generate_korean_response(prompt)
            print(f"답변: {response}")
        except Exception as e:
            print(f"오류 발생: {e}")
        
        print("-" * 50)
    
    print(f"\n✅ SOLAR-10.7B 한국어 파인튜닝 완료!")
    print(f"📁 모델 위치: {final_path}")
    print(f"🎯 훈련 데이터: {len(train_data):,}개 한국어 샘플")
    
else:
    print("데이터셋이 없어 모델 테스트를 건너뜁니다.")

## 8. (선택) Hugging Face 업로드 및 추론 테스트

훈련이 완료된 모델을 Hugging Face Hub에 업로드하고, 간단한 추론 테스트를 통해 성능을 확인합니다.


In [ ]:
def upload_and_test(model_path):
    # Hugging Face 로그인
    if config.hf_token:
        login(token=config.hf_token)
    
    # 모델 병합 및 업로드
    base_model = AutoModelForCausalLM.from_pretrained(config.base_model, torch_dtype=torch.bfloat16, device_map="auto")
    peft_model = PeftModel.from_pretrained(base_model, model_path)
    merged_model = peft_model.merge_and_unload()
    
    repo_name = f"{config.hf_username}/{config.model_name}"
    merged_model.push_to_hub(repo_name)
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.push_to_hub(repo_name)
    
    logger.info(f"✅ 모델을 {repo_name}에 업로드했습니다.")
    
    # 추론 테스트
    logger.info("🔍 추론 테스트 시작...")
    pipe = pipeline("text-generation", model=repo_name, torch_dtype=torch.bfloat16, device_map="auto")
    
    messages = [{"role": "user", "content": "창고에서 피킹 효율을 높이는 방법 3가지를 알려주세요."}]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_p=0.9)
    
    print("\n--- 추론 결과 ---")
    print(outputs[0]["generated_text"])
    print("-----------------")

# 훈련 완료 후 실행
# final_model_path = os.path.join(config.output_dir, "final")
# upload_and_test(final_model_path)
